In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing import image
import numpy as np 
import pandas as pd
from os import listdir
from PIL import Image as PImagea
from skimage.transform import resize
from random import shuffle
import cv2
import sklearn
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import glob
import tensorflow as tf
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications
from keras.applications.inception_v3 import preprocess_input
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from matplotlib.image import NonUniformImage
from google.colab.patches import cv2_imshow
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import decode_predictions
from tensorflow.keras.layers import Input


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
database = '/content/drive/MyDrive/PS I G3/Dataset/OuTex/Outex/Outex-TC-00010/000/'

data = []
labels = []

# classes
with open(database + "classes.txt", "r") as file:
    numClasses = int(file.readline())
    classes = {}
    
    for line in file:
        columns = line.split()
        classes[int(columns[1])] = columns[0]

# train
print("Training steps:")
with open(database + "test_org.txt", "r") as train:
    with open(database + "test_files_org.txt", "r") as trainFiles:
        numTrain = train.readline()
        print(numTrain)
        i = 0
        for pathFile in trainFiles:
            pathFile= pathFile.strip()
            pathFile = "/" + pathFile[1:]
            image=tf.keras.preprocessing.image.load_img(pathFile, color_mode="rgb", target_size= (448,448))
            image=np.array(image)
            data.append(image)
            print(i)
            i+=1
            label = int(train.readline().split()[1])
            labels.append(label)
            print()
            
data = np.array(data)
labels = np.array(labels)

Streaming output truncated to the last 5000 lines.
1340

1341

1342

1343

1344

1345

1346

1347

1348

1349

1350

1351

1352

1353

1354

1355

1356

1357

1358

1359

1360

1361

1362

1363

1364

1365

1366

1367

1368

1369

1370

1371

1372

1373

1374

1375

1376

1377

1378

1379

1380

1381

1382

1383

1384

1385

1386

1387

1388

1389

1390

1391

1392

1393

1394

1395

1396

1397

1398

1399

1400

1401

1402

1403

1404

1405

1406

1407

1408

1409

1410

1411

1412

1413

1414

1415

1416

1417

1418

1419

1420

1421

1422

1423

1424

1425

1426

1427

1428

1429

1430

1431

1432

1433

1434

1435

1436

1437

1438

1439

1440

1441

1442

1443

1444

1445

1446

1447

1448

1449

1450

1451

1452

1453

1454

1455

1456

1457

1458

1459

1460

1461

1462

1463

1464

1465

1466

1467

1468

1469

1470

1471

1472

1473

1474

1475

1476

1477

1478

1479

1480

1481

1482

1483

1484

1485

1486

1487

1488

1489

1490

1491

1492

1493

1494

1495

1496

1497

1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [6]:
# rows,cols,channels = np.shape(data[0])
# print('rows,cols,channels = ',rows,cols,channels)
rows,cols,channels = (448,448,3)
numClasses = 24

In [ ]:
X_train.shape

(3072, 448, 448, 3)

# x_5,x_5_3 (m+d)

In [23]:
# model_pretrained = tf.keras.applications.InceptionV3(weights="imagenet",include_top=False,input_shape=(rows,cols,3))
# model_pretrained.summary()

# inp = model_pretrained.input
# out = model_pretrained.layers[-2].output

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import tensorflow as tf

# create the base pre-trained model
model_pretrained = VGG16(weights='imagenet', include_top=False,input_shape=(rows,cols,3))
model_pretrained.summary()

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers except the last dense 
# layer for 11 classes that was just added.
for layer in model_pretrained.layers:
    layer.trainable = False

x_5 = model_pretrained.get_layer(index=-2).output
x_5_3 = model_pretrained.get_layer(index=-5).output
added_layer = tf.einsum('ijkm,ijkn->imn',x_5,x_5_3)
x = tf.reshape(added_layer,[-1,512*512])
print('Shape of phi_I after reshape', x.get_shape())

x = tf.divide(x,(784.0))  
print('Shape of phi_I after division', x.get_shape())

a = tf.multiply(tf.sign(x),tf.sqrt(tf.abs(x)+1e-12))
print('Shape of y_ssqrt', a.get_shape())

z = tf.nn.l2_normalize(a, dim=1)
print('Shape of z_l2', z.get_shape())


# x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# x = Dense(1024, activation='relu')(x)

# and a logistic layer -- let's say we have 24 classes
predictions = Dense(numClasses, activation='softmax')(z)



# this is the model we will train
bcnn_model = Model(inputs=model_pretrained.input, outputs=predictions)

# compile the model (should be done *after* setting layers to non-trainable)
bcnn_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

one_hot_labels = tf.keras.utils.to_categorical(labels, num_classes=numClasses)
bcnn_model.summary()
bcnn_model.fit(x=data,y=one_hot_labels,epochs=10,batch_size=32)
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.utils.vis_utils import plot_model
# # model = Sequential()
# # model.add(Dense(2, input_dim=1, activation='relu'))
# # model.add(Dense(1, activation='sigmoid'))
# plot_model(bcnn_model, show_shapes=True, show_layer_names=True)


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 448, 448, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 448, 448, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 448, 448, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 224, 224, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 224, 224, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 224, 224, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 112, 112, 128)     0     

AttributeError: ignored

Visualise the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.utils.vis_utils import plot_model
# model = Sequential()

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
one_hot_labels_test = tf.keras.utils.to_categorical(y_test, num_classes=24)
test_loss,test_acc = model.evaluate(X_test,  one_hot_labels_test, verbose=2)

print('\nTest accuracy:', test_acc*100,"%")

24/24 - 13s - loss: 0.0150 - accuracy: 1.0000

Test accuracy: 100.0 %


Fine tuning

In [ ]:
# model_pretrained = tf.keras.applications.InceptionV3(weights="imagenet",include_top=False,input_shape=(rows,cols,3))
# model_pretrained.summary()

# inp = model_pretrained.input
# out = model_pretrained.layers[-2].output

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import tensorflow as tf

# create the base pre-trained model
model_pretrained = VGG16(weights='imagenet', include_top=False,input_shape=(rows,cols,3))
model_pretrained.summary()

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers except the last dense 
# layer for 11 classes that was just added.
for layer in model_pretrained.layers[:15]:
    layer.trainable = False
for layer in model_pretrained.layers[15:]:
    layer.trainable = True

x_5 = model_pretrained.get_layer(index=-2).output
x_5_3 = model_pretrained.get_layer(index=-5).output
added_layer = tf.einsum('ijkm,ijkn->imn',x_5,x_5_3)
x = tf.reshape(added_layer,[-1,512*512])
print('Shape of phi_I after reshape', x.get_shape())

x = tf.divide(x,(784.0))  
print('Shape of phi_I after division', x.get_shape())

a = tf.multiply(tf.sign(x),tf.sqrt(tf.abs(x)+1e-12))
print('Shape of y_ssqrt', a.get_shape())

z = tf.nn.l2_normalize(a, dim=1)
print('Shape of z_l2', z.get_shape())


# x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# x = Dense(1024, activation='relu')(x)

# and a logistic layer -- let's say we have 24 classes
predictions = Dense(numClasses, activation='softmax')(z)



# this is the model we will train
model = Model(inputs=model_pretrained.input, outputs=predictions)

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
# y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
# y_test = np.asarray(y_test).astype('float32').reshape((-1,1))
one_hot_labels = tf.keras.utils.to_categorical(labels, num_classes=numClasses)
model.summary()
model.fit(x=data,y=one_hot_labels,epochs=10,batch_size=32)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 448, 448, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 448, 448, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 448, 448, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 224, 224, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 224, 224, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 224, 224, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 112, 112, 128)     0     

In [ ]:
one_hot_labels_test = tf.keras.utils.to_categorical(y_test, num_classes=24)
test_loss,test_acc = model.evaluate(X_test,  one_hot_labels_test, verbose=2)

print('\nTest accuracy:', test_acc*100,"%")

# x_5,x_5 (d+d)

In [ ]:
# model_pretrained = tf.keras.applications.InceptionV3(weights="imagenet",include_top=False,input_shape=(rows,cols,3))
# model_pretrained.summary()

# inp = model_pretrained.input
# out = model_pretrained.layers[-2].output

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import tensorflow as tf

# create the base pre-trained model
model_pretrained = VGG16(weights='imagenet', include_top=False,input_shape=(rows,cols,3))
model_pretrained.summary()

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers except the last dense 
# layer for 11 classes that was just added.
for layer in model_pretrained.layers:
    layer.trainable = False

x_5 = model_pretrained.get_layer(index=-2).output
x_5_3 = model_pretrained.get_layer(index=-5).output
added_layer = tf.einsum('ijkm,ijkn->imn',x_5,x_5)
x = tf.reshape(added_layer,[-1,512*512])
print('Shape of phi_I after reshape', x.get_shape())

x = tf.divide(x,(784.0))  
print('Shape of phi_I after division', x.get_shape())

a = tf.multiply(tf.sign(x),tf.sqrt(tf.abs(x)+1e-12))
print('Shape of y_ssqrt', a.get_shape())

z = tf.nn.l2_normalize(a, dim=1)
print('Shape of z_l2', z.get_shape())


# x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# x = Dense(1024, activation='relu')(x)

# and a logistic layer -- let's say we have 24 classes
predictions = Dense(numClasses, activation='softmax')(z)



# this is the model we will train
model = Model(inputs=model_pretrained.input, outputs=predictions)

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
# y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
# y_test = np.asarray(y_test).astype('float32').reshape((-1,1))
one_hot_labels = tf.keras.utils.to_categorical(labels, num_classes=numClasses)
model.summary()
model.fit(x=data,y=one_hot_labels,epochs=10,batch_size=32)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 448, 448, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 448, 448, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 448, 448, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 224, 224, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 224, 224, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 224, 224, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 112, 112, 128)     0     

In [ ]:
one_hot_labels_test = tf.keras.utils.to_categorical(y_test, num_classes=24)
test_loss,test_acc = model.evaluate(X_test,  one_hot_labels_test, verbose=2)

print('\nTest accuracy:', test_acc*100,"%")

24/24 - 13s - loss: 0.0354 - accuracy: 1.0000

Test accuracy: 100.0 %


Fine tuning

In [ ]:
# model_pretrained = tf.keras.applications.InceptionV3(weights="imagenet",include_top=False,input_shape=(rows,cols,3))
# model_pretrained.summary()

# inp = model_pretrained.input
# out = model_pretrained.layers[-2].output

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import tensorflow as tf

# create the base pre-trained model
model_pretrained = VGG16(weights='imagenet', include_top=False,input_shape=(rows,cols,3))
model_pretrained.summary()

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers except the last dense 
# layer for 11 classes that was just added.
for layer in model_pretrained.layers[:15]:
    layer.trainable = False
for layer in model_pretrained.layers[15:]:
    layer.trainable = True

x_5 = model_pretrained.get_layer(index=-2).output
x_5_3 = model_pretrained.get_layer(index=-5).output
added_layer = tf.einsum('ijkm,ijkn->imn',x_5,x_5)
x = tf.reshape(added_layer,[-1,512*512])
print('Shape of phi_I after reshape', x.get_shape())

x = tf.divide(x,(784.0))  
print('Shape of phi_I after division', x.get_shape())

a = tf.multiply(tf.sign(x),tf.sqrt(tf.abs(x)+1e-12))
print('Shape of y_ssqrt', a.get_shape())

z = tf.nn.l2_normalize(a, dim=1)
print('Shape of z_l2', z.get_shape())


# x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# x = Dense(1024, activation='relu')(x)

# and a logistic layer -- let's say we have 24 classes
predictions = Dense(numClasses, activation='softmax')(z)



# this is the model we will train
model = Model(inputs=model_pretrained.input, outputs=predictions)

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
# y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
# y_test = np.asarray(y_test).astype('float32').reshape((-1,1))
one_hot_labels = tf.keras.utils.to_categorical(labels, num_classes=numClasses)
model.summary()
model.fit(x=data,y=one_hot_labels,epochs=10,batch_size=32)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 448, 448, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 448, 448, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 448, 448, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 224, 224, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 224, 224, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 224, 224, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 112, 112, 128)     0     

In [ ]:
one_hot_labels_test = tf.keras.utils.to_categorical(y_test, num_classes=24)
test_loss,test_acc = model.evaluate(X_test,  one_hot_labels_test, verbose=2)

print('\nTest accuracy:', test_acc*100,"%")

24/24 - 13s - loss: 1.3144e-04 - accuracy: 1.0000

Test accuracy: 100.0 %
